#### เตรียมพร้อม, ติดตั้ง library, ตั้งค่าไพล์ .env

In [2]:
%cp --update=none .env.example .env

cp: option '--update' doesn't allow an argument
Try 'cp --help' for more information.


In [3]:
%pip install -r requirements.txt --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [7]:
import os
import json
from flask import Flask, request, abort
from collections import deque

import google.generativeai as genai

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

In [8]:
LINE_CHANNEL_SECRET = os.environ['LINE_CHANNEL_SECRET']
LINE_CHANNEL_TOKEN = os.environ['LINE_CHANNEL_TOKEN']
GENAI_API_KEY = os.environ['GENAI_API_KEY']

In [9]:
genai.configure(api_key=GENAI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash-002")

In [10]:
app = Flask(__name__)

configuration = Configuration(access_token=LINE_CHANNEL_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

In [11]:
MAX_HISTORY = 10
conversation_history = {}

In [12]:
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel")
        abort(400)
    
    return 'OK'

In [13]:
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        user_id = event.source.user_id
        user_message = event.message.text
        
        if user_message.lower() == "/clear":
            if user_id in conversation_history:
                conversation_history[user_id].clear
            response_text = ""
        else:
            if user_id not in conversation_history:
                conversation_history[user_id] = deque(maxlen=MAX_HISTORY)
        
        conversation_history[user_id].append(
            {"role": "user", "parts": [user_message]}
        )
        
        conversations = list(conversation_history[user_id])
        
        gemini_resp = model.generate_content(conversations)
        resp_text = gemini_resp.text

        conversation_history[user_id].append(
            {"role": "model", "parts": [resp_text]}
        )
        
        dump_conversation_history = json.dumps(list(conversation_history[user_id]))
        app.logger.info(f"Conversation history for user {user_id}: {dump_conversation_history}")
        
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=resp_text)]
            )
        )

In [14]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
